# Neural Network - Optimization & Regularization

1. Train two models:
   - **Neural Network (MLP): Default**
   - **Neural Network (MLP): Optimization**
2. Compare performance using clear metrics and visualizations.
3. Highlight **what leaders should focus on** in evaluation and decision‑making.

In [ ]:
import pandas as pd, numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt, warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, roc_curve, precision_recall_curve
from sklearn.calibration import calibration_curve
from scipy.io import arff

rng = default_rng(0)
warnings.filterwarnings("ignore")

## Dataset Description
- Author: Paulo Cortez, Sérgio Moro 
- Source: UCI 
- Cite: S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

### Background
The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (or not) subscribed.

The classification goal is to predict if the client will subscribe a term deposit (variable y).

### Features:
**bank client data:**
- age (numeric)
- job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur", "student","blue-collar","self-employed","retired","technician","services")
- marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
- education (categorical: "unknown","secondary","primary","tertiary")
- default: has credit in default? (binary: "yes","no")
- balance: average yearly balance, in euros (numeric)
- housing: has housing loan? (binary: "yes","no")
- loan: has personal loan? (binary: "yes","no")
- **noise_1-20**: random noise columns (intentionally added to simulate real-world scenario)

**output variable (desired target):**
- y - has the client subscribed a term deposit? (binary: "yes","no")

In [ ]:
# Load Dataset
!wget -q https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/NeuralNetwork/phpkIxskf.arff
file_name = "phpkIxskf.arff"
data, meta = arff.loadarff(file_name)
df = pd.DataFrame(data)
# Rename columns
col_map = {
    "V1": "age",
    "V2": "job",
    "V3": "marital",
    "V4": "education",
    "V5": "default",
    "V6": "balance",
    "V7": "housing",
    "V8": "loan",
    "Class": "subscribed"
}

cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']

df = df.rename(columns=col_map)
df = df[col_map.values()]

# Convert data types
def decode_bytes(val):
    if isinstance(val, (bytes, bytearray)):
        try:
            return val.decode('utf-8')  # decode to string
        except:
            return str(val)  # fallback to str
    if isinstance(val, str) and val.startswith("b'"):  # string that looks like b'...'
        return val.strip("b'").strip("'")
    return val

df = df.applymap(decode_bytes)

# Convert target (binary)
df["subscribed"] = df["subscribed"].replace({"1": 1, "2": 0})

# Intentional data tweeking (Educational purpose)
# Add random noise columns
np.random.seed(42)
for i in range(20):
    df[f"noise_{i}"] = np.random.randn(len(df))

# Ensure numeric columns are numeric
num_cols = list(set(df.columns) - set(cat_cols + ['subscribed']))
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

print(df.head())

## Train/Test split
Hold out 20% of data for final evaluation.

In [ ]:
X = df.drop(columns=['subscribed'])
y = df['subscribed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

## Neural Network (MLP - Default)
A simple feedforward network (multi‑layer perceptron). More flexible but less interpretable.

In [ ]:
mlp = Pipeline([
    ('pre', pre),
    ('clf', MLPClassifier(hidden_layer_sizes=(16, 8)))
])
mlp.fit(X_train, y_train)
probs_nn = mlp.predict_proba(X_test)[:,1]

auc_nn = roc_auc_score(y_test, probs_nn)
pr_nn = average_precision_score(y_test, probs_nn)
brier_nn = brier_score_loss(y_test, probs_nn)

result = pd.DataFrame({
    "NN result":{
        "roc_auc_score": auc_nn,
        "average_precision_score": pr_nn,
        "brier_score_loss": brier_nn
    }
})

display(result.round(3))

# Training Loss curve
clf = mlp.named_steps["clf"]
plt.figure(figsize=(6,4))
plt.plot(clf.loss_curve_)
plt.xlabel("epoch"); plt.ylabel("loss")
plt.title("MLP training loss curve")
plt.tight_layout(); plt.show()

## Neural Network (MLP) - Hyperparameter Tuning
A simple feedforward network (multi‑layer perceptron) with hyperparameter tunings.

In [ ]:
mlp_op = Pipeline([
    ('pre', pre),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(16, 8),
        activation='relu',      # 'relu', 'identity', 'logistic', 'tanh'
        solver='adam',          # 'lbfgs', 'sgd', 'adam'
        alpha=0.01,
        batch_size=50,
        learning_rate='adaptive',
        max_iter=3000,
        shuffle=True,
        early_stopping=True,
        n_iter_no_change=50,
        validation_fraction=0.2
        ))
])
mlp_op.fit(X_train, y_train)
probs_op = mlp_op.predict_proba(X_test)[:,1]

auc_op = roc_auc_score(y_test, probs_op)
pr_op = average_precision_score(y_test, probs_op)
brier_op = brier_score_loss(y_test, probs_op)

result = pd.DataFrame({
    "NN result":{
        "roc_auc_score": auc_op,
        "average_precision_score": pr_op,
        "brier_score_loss": brier_op
    }
})

display(result.round(3))

# Training Loss curve
clf_op = mlp_op.named_steps["clf"]
plt.figure(figsize=(6,4))
plt.plot(clf_op.loss_curve_)
plt.xlabel("epoch"); plt.ylabel("loss")
plt.title("MLP training loss curve")
plt.tight_layout(); plt.show()

## NN Default vs. Optimized
- **ROC‑AUC** (ranking quality): How well the model ranks likely buyers above non-buyers. (higher=better)
- **PR‑AUC** (precision‑recall for rare positives): How well the model finds true buyers among the top predictions when buyers are rare. (higher=better)
- **Brier score** (calibration): How well the predicted probabilities match reality (calibration). (lower=better)

In [ ]:
# ROC curves
fpr_nn, tpr_nn, _ = roc_curve(y_test, probs_nn)
fpr_op, tpr_op, _ = roc_curve(y_test, probs_op)

plt.figure(figsize=(6,5))
plt.plot(fpr_nn, tpr_nn, label=f"Neural Net (AUC={auc_nn:.3f})")
plt.plot(fpr_op, tpr_op, label=f"Neural Net(Op) (AUC={auc_op:.3f})")
plt.plot([0,1],[0,1],'--', color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()

# Precision-Recall curves
prec_nn, rec_nn, _ = precision_recall_curve(y_test, probs_nn)
prec_op, rec_op, _ = precision_recall_curve(y_test, probs_op)

plt.figure(figsize=(6,5))
plt.plot(rec_nn, prec_nn, label=f"Neural Net (PR-AUC={pr_nn:.3f})")
plt.plot(rec_op, prec_op, label=f"Neural Net(op) (PR-AUC={pr_op:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve Comparison")
plt.legend()
plt.show()

# Calibration (reliability) curve
prob_true_nn, prob_pred_nn = calibration_curve(y_test, probs_nn, n_bins=12, strategy='quantile')
prob_true_op, prob_pred_op = calibration_curve(y_test, probs_op, n_bins=12, strategy='quantile')

plt.figure(figsize=(6,5))
plt.plot(prob_pred_nn, prob_true_nn, label=f"Neural Net (Brier={brier_nn:.3f})")
plt.plot(prob_pred_op, prob_true_op, label=f"Neural Net(Op) (Brier={brier_op:.3f})")
plt.plot([0,1],[0,1],'--')
plt.xlabel("Predicted probability (bin mean)")
plt.ylabel("Observed frequency")
plt.title("Calibration")
plt.legend()
plt.show()

results = pd.DataFrame({
    "Neural Net": {
        "ROC-AUC (higher=better)": auc_nn,
        "PR-AUC (higher=better)": pr_nn,
        "Brier (lower=better)": brier_nn
    },
    "Neural Net(Op)": {
        "ROC-AUC (higher=better)": auc_op,
        "PR-AUC (higher=better)": pr_op,
        "Brier (lower=better)": brier_op
    }
})

display(results.T.round(3))